In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns


In [3]:
df_quality1 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/quality1.csv')
school15 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/school15.csv')
school25 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/school25.csv')
df_quality2 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/quality2.csv')

In [5]:
df_quality1.columns

Index(['SHCODE', 'WBCTRY', 'COUNTRY', 'TEAPRI60', 'TEAPRI65', 'TEAPRI70',
       'TEAPRI75', 'TEAPRI80', 'TEAPRI85', 'TEAPRI90', 'TEASEC60', 'TEASEC65',
       'TEASEC70', 'TEASEC75', 'TEASEC80', 'TEASEC85', 'TEASEC90', 'GEEPRI60',
       'GEEPRI65', 'GEEPRI70', 'GEEPRI75', 'GEEPRI80', 'GEEPRI85', 'GEEPRI90',
       'GEESEC60', 'GEESEC65', 'GEESEC70', 'GEESEC75', 'GEESEC80', 'GEESEC85',
       'GEESEC90', 'SHPUPP60', 'SHPUPP65', 'SHPUPP70', 'SHPUPP75', 'SHPUPP80',
       'SHPUPP85', 'SHPUPP90', 'SHPUPS60', 'SHPUPS65', 'SHPUPS70', 'SHPUPS75',
       'SHPUPS80', 'SHPUPS85', 'SHPUPS90', 'REPPRI65', 'REPPRI70', 'REPPRI75',
       'REPPRI80', 'REPPRI85', 'REPPRI90', 'REPSEC70', 'REPSEC75', 'REPSEC80',
       'REPSEC85', 'REPSEC90', 'SCHDAY', 'SCHTIME', 'SALARP60', 'SALARP65',
       'SALARP70', 'SALARP75', 'SALARP80', 'SALARP85', 'SALARP90', 'SHSALP60',
       'SHSALP65', 'SHSALP70', 'SHSALP75', 'SHSALP80', 'SHSALP85', 'SHSALP90',
       'DROP70', 'DROP75', 'DROP80', 'DROP85', 'DROP90', 'CO

In [6]:
df_quality2.columns

Index(['SHCODE', 'COUNTRY', 'M_64_13', 'M_64_FS', 'M_8283_13', 'M_8283_FS',
       'M_88_13', 'M_9091_10', 'M_9091_14', 'M_9398_13', 'S_7072_10',
       'S_7072_14', 'S_7072_FS', 'S_84_10', 'S_84_13', 'S_84_FS', 'S_88_13',
       'S_9091_9', 'S_9091_13', 'S_9398_13', 'R_7072_13', 'R_9091_9',
       'R_9091_13'],
      dtype='object')

In [7]:
school15.columns

Index(['SHCODE', 'COUNTRY', 'YEAR', 'N015', 'PRI15', 'PRIC15', 'SEC15',
       'SECC15', 'HIGH15', 'HIGHC15', 'TYR15', 'PYR15', 'SYR15', 'HYR15'],
      dtype='object')

In [8]:
school25.columns

Index(['SHCODE', 'COUNTRY', 'YEAR', 'N025', 'PRI25', 'PRIC25', 'SEC25',
       'SECC25', 'HIGH25', 'HIGHC25', 'TYR25', 'PYR25', 'SYR25', 'HYR25'],
      dtype='object')

selecting the feature variaables

In [70]:

df_quality1_selected = df_quality1[
    ['SHCODE', 'COUNTRY', 'DROP70', 'DROP75', 'DROP80', 'DROP85', 'DROP90',
     'REPPRI65', 'REPPRI70', 'REPPRI75', 'REPPRI80', 'REPPRI85', 'REPPRI90',
     'REPSEC70', 'REPSEC75', 'REPSEC80', 'REPSEC85', 'REPSEC90',
     'SALARP60', 'SALARP65', 'SALARP70', 'SALARP75', 'SALARP80', 'SALARP85', 'SALARP90',
     'SHPUPP60', 'SHPUPP65', 'SHPUPP70', 'SHPUPP75', 'SHPUPP80', 'SHPUPP85', 'SHPUPP90',
     'SHPUPS60', 'SHPUPS65', 'SHPUPS70', 'SHPUPS75', 'SHPUPS80', 'SHPUPS85', 'SHPUPS90'
    ]
]


final_data = df_quality1_selected

In [71]:
final_data.fillna(method='ffill', inplace=True)

data_df=pd.DataFrame(final_data)
data_df.head()

<ipython-input-71-c6ddcd87673e>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_data.fillna(method='ffill', inplace=True)
<ipython-input-71-c6ddcd87673e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data.fillna(method='ffill', inplace=True)


,SHCODE,COUNTRY,DROP70,DROP75,DROP80,DROP85,DROP90,REPPRI65,REPPRI70,REPPRI75,...,SHPUPP80,SHPUPP85,SHPUPP90,SHPUPS60,SHPUPS65,SHPUPS70,SHPUPS75,SHPUPS80,SHPUPS85,SHPUPS90
0,1,Algeria,31.3,28.8,23.5,9.8,10.0,NaN,NaN,13.0,...,8.600000,NaN,13.3,NaN,75.199997,48.900002,59.200001,23.000000,NaN,31.5
1,2,Angola,66.0,66.0,66.0,66.0,66.0,NaN,NaN,13.0,...,8.600000,NaN,13.3,NaN,75.199997,48.900002,59.200001,23.000000,NaN,31.5
2,3,Benin,64.0,64.0,64.0,64.0,60.0,22.0,17.0,22.0,...,18.200001,NaN,13.3,NaN,123.800003,101.099998,39.400002,40.200001,NaN,31.5
3,4,Botswana,50.4,18.2,26.7,10.7,20.0,0.0,0.0,3.0,...,11.200000,9.200000,13.3,147.100006,224.199997,97.900002,55.500000,51.400002,63.799999,31.5
4,5,Burkina Faso,51.0,38.3,25.1,26.2,29.0,18.0,0.0,3.0,...,22.000000,16.299999,13.3,147.100006,224.199997,97.900002,174.600006,98.900002,57.099998,31.5


lets use 3 models and compare the outcomes of all of them to see which one has the best prediction



In [78]:
final_data['avg_drop_rate'] = final_data[['DROP70', 'DROP75', 'DROP80', 'DROP85', 'DROP90']].mean(axis=1)

<ipython-input-78-acd961bc605f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['avg_drop_rate'] = final_data[['DROP70', 'DROP75', 'DROP80', 'DROP85', 'DROP90']].mean(axis=1)


In [80]:

features = [
     'REPSEC70', 'REPSEC75', 'REPSEC80', 'REPSEC85', 'REPSEC90',
     'SALARP60', 'SALARP65', 'SALARP70', 'SALARP75', 'SALARP80', 'SALARP85', 'SALARP90',
     'SHPUPP60', 'SHPUPP65', 'SHPUPP70', 'SHPUPP75', 'SHPUPP80', 'SHPUPP85', 'SHPUPP90',
     'SHPUPS60', 'SHPUPS65', 'SHPUPS70', 'SHPUPS75', 'SHPUPS80', 'SHPUPS85', 'SHPUPS90'
    ]


dic = dict()

for feature in features:
    dic[feature] = final_data[feature].corr(final_data['avg_drop_rate'])

corrs = pd.Series(dic, name='Correlation')
corrs.sort_values()


,Correlation
SALARP85,-0.508794
SALARP80,-0.443995
SALARP90,-0.438711
SALARP75,-0.417437
SALARP65,-0.375563
SALARP70,-0.364432
SHPUPP90,-0.318189
SHPUPP85,-0.299350
SALARP60,-0.253682
SHPUPP80,-0.225238


using salary as the best features

In [105]:

best_features = ['SALARP85', 'SALARP80', 'SALARP90', 'SALARP75', 'SALARP65', 'SALARP70']

X = final_data[best_features]
y = final_data['avg_drop_rate']



In [95]:


best_features = ['SALARP85', 'SALARP80', 'SALARP90', 'SALARP75', 'SALARP65', 'SALARP70']

X = final_data[best_features]

y1 = final_data['avg_drop_rate']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), best_features)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])




In [96]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y1, test_size=0.2, random_state=42)

model.fit(X_train_d, y_train_d)
y_pred_d = model.predict(X_test_d)
print(f"Predicted Drop-out Rates: {y_pred_d[:5]}")


Predicted Drop-out Rates: [32.474289   28.08670791 30.98612443 32.13580033 33.53278794]


using linear regressor

In [98]:
mse = mean_squared_error(y_test_d, y_pred_d)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 21.538492532344392


using ratio of gpero for features

In [106]:


best_features = ['SHPUPP90', 'SHPUPP85', 'SALARP60', 'SALARP75', 'SHPUPP80', 'SHPUPP75']

X = final_data[best_features]

y1 = final_data['avg_drop_rate']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), best_features)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])




In [107]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y1, test_size=0.2, random_state=42)

model.fit(X_train_d, y_train_d)
y_pred_d = model.predict(X_test_d)
print(f"Predicted Drop-out Rates: {y_pred_d[:5]}")


Predicted Drop-out Rates: [32.00133324 22.96541274 30.69427475 29.03913043 31.85453414]


In [108]:
mse = mean_squared_error(y_test_d, y_pred_d)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 23.65230597538617


From both these model we can understand that the the above using salary was a better feature to predict the models as we can see that the rmse was much better using salary